# 4: [GLIP](https://github.com/microsoft/GLIP) inference


> Made for running on Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install CUDA 10.2; newer versions of CUDA may fail
!apt-get update -y
!apt-get --purge remove "*cublas*" "cuda*" "nsight*" 
!nvcc --version

!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
!apt-key add /var/cuda-repo-10-2-local-10.2.89-440.33.01/7fa2af80.pub
!apt-get update
!apt-get -y install cuda

In [ ]:
# Install Environments. This will take a few minutes. Please be patient ;)
! nvidia-smi
! git clone https://github.com/microsoft/GLIP.git
%cd GLIP
! git checkout c663d9db8a503e04c6b76cd2e14152bab775d28a
! pip install torch==1.9.0 torchvision  torchaudio
! pip install einops shapely timm yacs tensorboardX ftfy prettytable pymongo
! pip install transformers
! python setup.py build develop --user
! mkdir MODEL

In [ ]:
import pickle

import numpy as np

from PIL import Image
from maskrcnn_benchmark.config import cfg
from maskrcnn_benchmark.engine.predictor_glip import GLIPDemo

In [ ]:
!cp /content/drive/MyDrive/BCUFR/dataset_for_phrase_grounding/dataset_for_phrase_grounding.zip dataset.zip &&  unzip -q dataset.zip 
dataset = pickle.load(open('dataset.p','rb'))
!cp  /content/drive/MyDrive/BCUFR/GLIP/glip_large_model.pth  /content/GLIP/MODEL/glip_large_model.pth 

In [ ]:
# IF NO WEIGHTS ON DRIVE
# !wget https://penzhanwu2bbs.blob.core.windows.net/data/GLIPv1_Open/models/glip_tiny_model_o365_goldg_cc_sbu.pth -O MODEL/glip_tiny_model_o365_goldg_cc_sbu.pth
# config_file = "configs/pretrain/glip_Swin_T_O365_GoldG.yaml"
# weight_file = "MODEL/glip_tiny_model_o365_goldg_cc_sbu.pth"

# Use this command to evaluate the GLPT-L model
# ! wget https://penzhanwu2bbs.blob.core.windows.net/data/GLIPv1_Open/models/glip_large_model.pth -O MODEL/glip_large_model.pth

# IF WEIGHTS UPLOADED ON DRIVE
config_file = "configs/pretrain/glip_Swin_L.yaml"
weight_file = "/content/drive/MyDrive/BCUFR/GLIP/glip_large_model.pth"

# update the config options with the config file
# manual override some options
cfg.local_rank = 0
cfg.num_gpus = 1
cfg.merge_from_file(config_file)
cfg.merge_from_list(["MODEL.WEIGHT", weight_file])
cfg.merge_from_list(["MODEL.DEVICE", "cuda"])

In [ ]:
glip_demo = GLIPDemo(
    cfg,
    min_image_size=800,
    confidence_threshold=0.7,
    show_mask_heatmaps=False
)

Now let's inference on every (image, expression) in the dataset

In [ ]:
detection_results = []
for elem in dataset:
  pil_image = Image.open('img/' + elem["filename"]).convert('RGB')
  image = np.array(pil_image)[:, :, [2, 1, 0]]
  caption = elem['title']['preprocessed']
  if caption == 'actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor, actor':
    caption = 'actors'
  result, predictions = glip_demo.run_on_web_image(image, caption)
  # imshow(result, caption)
  # plt.show()
  detection_results.append([predictions, glip_demo.entities])

In [ ]:
pickle.dump(detection_results, open('/content/drive/MyDrive/BCUFR/GLIP_title.p','wb'))